In [46]:
import pandas as pd
from datetime import datetime, timedelta 
import datetime

datelist = pd.date_range('2021-03-01 08:00:00', periods=1).tolist()

In [43]:
for d in datelist:
    print(d, d + timedelta(days=-1))

2021-03-01 08:00:00 2021-02-28 08:00:00


In [40]:
import requests
import sqlite3
import praw
from credentials import credentials

client_id, client_secret,password, user_agent, username = credentials

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    password=password,
    user_agent=user_agent,
    username=username,
)

print(reddit.user.me())

JustAnotherCogg


In [ ]:
top_posts = reddit.subreddit('worldnews').top(time_filter='year',limit=100000)

#Get all unique columns
kolonas = ['moment', 'daily_rank']   
for post in top_posts:
     for m in vars(post):
            if m not in kolonas:
                kolonas.append(m)
                
sodiena = str("ER20202021")

createtable = ""
columnnames = ""
insertintotable = ""

#Create strings for SQL injection
z = len(kolonas)
for k in kolonas:
    if z > 1:
        createtable+= str(k + " TEXT,")
        columnnames += str(f"'{k}', ")
        insertintotable += str("?,")
        
    else:
        createtable+= str(k + " TEXT")
        columnnames += str(k)
        insertintotable += str("?")
    z+= -1
    
#Drop and create the todays table
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()
a = f"cursor.executescript('DROP TABLE IF EXISTS {sodiena}')"
b = f"cursor.execute('''CREATE TABLE {sodiena} ({createtable} )''')"

eval(a)
eval(b)
    
#An actual webscraping
top_posts = reddit.subreddit('worldnews').top(time_filter='year',limit=100000)
i = 1
for post in top_posts:    
    atbilde =[]
    atbilde.append(str(datetime.datetime.now()))
    atbilde.append(str(i))
    for j in kolonas[2:]:
        try:
            atbilde.append(str(eval("post."+j)))
        except:
            atbilde.append("")
    
    connection = sqlite3.connect('MSC.db')
    cursor = connection.cursor()
    sql2 = f'cursor.execute("insert INTO  {sodiena} ({columnnames}) VALUES ({insertintotable})", {atbilde})'
    eval(sql2)
    connection.commit()
    connection.close()
    
    i +=1
    
print("done")

In [ ]:
connection = sqlite3.connect('MSC.db')
cursor = connection.cursor()

sql = (f"Select * from {sodiena}")
z = pd.read_sql_query(sql,connection)
z.shape